In [17]:
import os
import tarfile
import pandas as pd
import zipfile
import shutil
import numpy as np
from google.cloud import storage
import tensorflow as tf
from tensorflow import keras

# User input for image file path  
copy path of a specific tar file from the WellImages storage_4:  
> console.cloud.google.com/storage/browser/storage_4/WellImages

In [2]:
### user input of path containing the images to be assessed 
user_input = input()

 gs://storage_4/WellImages/170.tar


In [18]:
### get current working directory
cwd = os.getcwd()
cwd

'/home/jupyter/Prediction'

In [4]:
### copy tar file from storage_4 to VM
!gsutil -m cp "$user_input" "$cwd"

Copying gs://storage_4/WellImages/170.tar...
- [1/1 files][190.0 MiB/190.0 MiB] 100% Done  17.4 MiB/s ETA 00:00:00           
Operation completed over 1 objects/190.0 MiB.                                    


In [5]:
### get tar file name and folder location
tar_file = user_input.split(os.sep)[-1]
tar_folder = tar_file[:-4]

# Extract tar file images

In [6]:
### unzip tar file 
fname = cwd + os.sep + tar_file
if fname.endswith("tgz"):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()

# Load the pre-trained model

In [19]:
### unizp the pre-trained model
pre_trained_model = "Inception_Pre-trained_Model.zip"
input_zip_path = cwd + os.sep + pre_trained_model

with zipfile.ZipFile(input_zip_path, 'r') as zip_ref:
    zip_ref.extractall(cwd) #target zip path = current working direcotry

In [8]:
#make sure to load the correct model in case you reset the runtime
new_model = tf.keras.models.load_model(cwd +"/Inception_Pre-trained_Model")

# Create Dataframe containing images complete path
> currently only collecting sharp images (filename contains "ef")   
> could be improved to predict on blurry images (needs training on blurry images first)

In [9]:
### read extracted tar folder and collect images filenames and complete path
### this DF is only a source of information for the predicting loop
### after predictions a new DF will be returned

fname = cwd + os.sep + tar_folder
path_list = []
name_list = []
for subdir, dirs, files in os.walk(fname):
    for filename in files:
        filepath = subdir + os.sep + filename
        if (filepath.endswith(".jpg") or filepath.endswith(".JPG")) and ("ef" in filename):
            path_list.append(filepath)
            name_list.append(filename)

In [10]:
df = pd.DataFrame(path_list, name_list).reset_index()
df.columns = ["file_name", "file_path"]
df = df.set_index("file_path")

# Predict using pre-trained model and file paths in Data Frame

In [11]:
%%time
classes = ["crystal", "no_crystal"]
IMG_SIZE = 224
predictions = {}

for path in df.index:
    img = keras.preprocessing.image.load_img(path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)/255
    img_array = tf.expand_dims(img_array, 0)
    pred = new_model.predict(img_array)
    predictions[path] = ((path.split(os.sep)[-1]), 
                         classes[np.argmax(pred)],
                         ("confidance of {:.2f}%".format(100 * np.max(pred))))

CPU times: user 28.9 s, sys: 1.89 s, total: 30.8 s
Wall time: 30.6 s


In [12]:
### a Data Frame is returned containing:$
### File Path, File Name, the model Prediction and the Prediction Confidence Level.
df_pred = pd.DataFrame.from_dict(predictions, orient="index")
df_pred.columns = ["File_Name", "Prediction", "Confidence"]

,File_Name,Prediction,Confidence
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_67/profileID_1/d2_r422734_ef.jpg,d2_r422734_ef.jpg,no_crystal,confidance of 99.41%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_67/profileID_1/d1_r422733_ef.jpg,d1_r422733_ef.jpg,no_crystal,confidance of 99.99%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_6/profileID_1/d2_r422612_ef.jpg,d2_r422612_ef.jpg,no_crystal,confidance of 95.28%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_6/profileID_1/d1_r422611_ef.jpg,d1_r422611_ef.jpg,no_crystal,confidance of 100.00%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_8/profileID_1/d1_r422615_ef.jpg,d1_r422615_ef.jpg,no_crystal,confidance of 99.76%
...,...,...,...
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_17/profileID_1/d1_r422633_ef.jpg,d1_r422633_ef.jpg,no_crystal,confidance of 99.90%
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_95/profileID_1/d1_r422789_ef.jpg,d1_r422789_ef.jpg,no_crystal,confidance of 99.70%
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_95/profileID_1/d2_r422790_ef.jpg,d2_r422790_ef.jpg,crystal,confidance of 100.00%
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_82/profileID_1/d1_r422763_ef.jpg,d1_r422763_ef.jpg,no_crystal,confidance of 100.00%


In [13]:
### return list of images predicted as "crystal"
df_pred[df_pred['Prediction'] == "crystal"]        

,File_Name,Prediction,Confidence
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_16/profileID_1/d1_r422631_ef.jpg,d1_r422631_ef.jpg,crystal,confidance of 96.57%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_94/profileID_1/d2_r422788_ef.jpg,d2_r422788_ef.jpg,crystal,confidance of 99.71%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_18/profileID_1/d2_r422636_ef.jpg,d2_r422636_ef.jpg,crystal,confidance of 99.24%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_54/profileID_1/d1_r422707_ef.jpg,d1_r422707_ef.jpg,crystal,confidance of 98.55%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_72/profileID_1/d1_r422743_ef.jpg,d1_r422743_ef.jpg,crystal,confidance of 99.99%
...,...,...,...
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_24/profileID_1/d1_r422647_ef.jpg,d1_r422647_ef.jpg,crystal,confidance of 99.61%
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_24/profileID_1/d2_r422648_ef.jpg,d2_r422648_ef.jpg,crystal,confidance of 94.10%
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_36/profileID_1/d2_r422672_ef.jpg,d2_r422672_ef.jpg,crystal,confidance of 100.00%
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_17/profileID_1/d2_r422634_ef.jpg,d2_r422634_ef.jpg,crystal,confidance of 77.87%


In [14]:
### return list of images predicted as "no_crystal"
df_pred[df_pred['Prediction'] == "no_crystal"]        

,File_Name,Prediction,Confidence
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_67/profileID_1/d2_r422734_ef.jpg,d2_r422734_ef.jpg,no_crystal,confidance of 99.41%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_67/profileID_1/d1_r422733_ef.jpg,d1_r422733_ef.jpg,no_crystal,confidance of 99.99%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_6/profileID_1/d2_r422612_ef.jpg,d2_r422612_ef.jpg,no_crystal,confidance of 95.28%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_6/profileID_1/d1_r422611_ef.jpg,d1_r422611_ef.jpg,no_crystal,confidance of 100.00%
/home/jupyter/Prediction/170/plateID_3170/batchID_21083/wellNum_8/profileID_1/d1_r422615_ef.jpg,d1_r422615_ef.jpg,no_crystal,confidance of 99.76%
...,...,...,...
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_36/profileID_1/d1_r422671_ef.jpg,d1_r422671_ef.jpg,no_crystal,confidance of 93.85%
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_17/profileID_1/d1_r422633_ef.jpg,d1_r422633_ef.jpg,no_crystal,confidance of 99.90%
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_95/profileID_1/d1_r422789_ef.jpg,d1_r422789_ef.jpg,no_crystal,confidance of 99.70%
/home/jupyter/Prediction/170/plateID_3170/batchID_21070/wellNum_82/profileID_1/d1_r422763_ef.jpg,d1_r422763_ef.jpg,no_crystal,confidance of 100.00%


# Delete .tar file and extracted folder

In [15]:
### paths to be deleted
dir_tar = cwd + os.sep + tar_file
dir_folder = cwd + os.sep + tar_folder

### removing tar file and extracted directory 
try:
    os.unlink(dir_tar)
    shutil.rmtree(dir_folder, ignore_errors = False)
except:
    pass

# Delete extracted model folder

In [16]:
# specify the directories to delete within the location
pre_trained_model_folder = pre_trained_model[:-4]
folder_to_del = cwd + os.sep + pre_trained_model_folder

# removing directory 
shutil.rmtree(folder_to_del, ignore_errors = False)